In [1]:
# Create SparkSession from builder
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.functions import col

#config session
spark = SparkSession.builder.master("local[*]").appName("IPIP-FFM")\
        .config("spark.driver.extraClassPath", "/home/user1664/Desktop/JDBC/mysql-connector-java-8.0.30.jar")\
        .getOrCreate()

23/08/15 00:00:15 WARN Utils: Your hostname, user1664-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/08/15 00:00:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/08/15 00:00:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df = spark.read.option("header",True).option("delimiter", "\t").csv("IPIP-FFM-data-8Nov2018/data-final.csv")

#Transform
df=df.coalesce(1).withColumn("UniqueID", functions.monotonically_increasing_id())
cols=['UniqueID','EXT1','EXT2', 'EXT3', 'EXT4', 'EXT5', 'EXT6', 'EXT7', 'EXT8', 'EXT9', 'EXT10', 'EST1', 'EST2', 'EST3', 'EST4', 'EST5', 'EST6', 'EST7', 'EST8', 'EST9', 'EST10', 'AGR1', 'AGR2', 'AGR3', 'AGR4', 'AGR5', 'AGR6', 'AGR7', 'AGR8', 'AGR9', 'AGR10', 'CSN1', 'CSN2', 'CSN3', 'CSN4', 'CSN5', 'CSN6' ,'CSN7', 'CSN8', 'CSN9', 'CSN10', 'OPN1', 'OPN2', 'OPN3', 'OPN4', 'OPN5', 'OPN6', 'OPN7', 'OPN8', 'OPN9', 'OPN10']
df=df.select([col(c).cast("int") for c in cols])

df_ext =  df.select("UniqueID","EXT1","EXT2","EXT3","EXT4","EXT5","EXT6","EXT7","EXT8","EXT9","EXT10")
df_arg =  df.select("UniqueID","AGR1","AGR2","AGR3","AGR4","AGR5","AGR6","AGR7","AGR8","AGR9","AGR10")
df_csn =  df.select("UniqueID","CSN1","CSN2","CSN3","CSN4","CSN5","CSN6","CSN7","CSN8","CSN9","CSN10")
df_opn =  df.select("UniqueID","OPN1","OPN2","OPN3","OPN4","OPN5","OPN6","OPN7","OPN8","OPN9","OPN10")

In [3]:
#duplicate check on UniqueID
df.groupBy("UniqueID").count().where("count > 1").drop("count").show()

+--------+
|UniqueID|
+--------+
+--------+



In [15]:
df_opn.show()
df_opn.printSchema()
"Total count= "+ str(df_opn.count())

+--------+----+----+----+----+----+----+----+----+----+-----+
|UniqueID|OPN1|OPN2|OPN3|OPN4|OPN5|OPN6|OPN7|OPN8|OPN9|OPN10|
+--------+----+----+----+----+----+----+----+----+----+-----+
|       0|   5|   1|   4|   1|   4|   1|   5|   3|   4|    5|
|       1|   1|   2|   4|   2|   3|   1|   4|   2|   5|    3|
|       2|   5|   1|   2|   1|   4|   2|   5|   3|   4|    4|
|       3|   4|   2|   5|   2|   3|   1|   4|   4|   3|    3|
|       4|   5|   1|   5|   1|   5|   1|   5|   3|   5|    5|
|       5|   5|   1|   5|   1|   3|   1|   5|   4|   5|    2|
|       6|   5|   2|   4|   3|   4|   1|   5|   5|   4|    4|
|       7|   4|   1|   3|   1|   5|   1|   4|   3|   4|    5|
|       8|   4|   1|   5|   1|   4|   1|   5|   2|   5|    5|
|       9|   3|   1|   3|   1|   3|   3|   4|   3|   3|    3|
|      10|   5|   1|   5|   1|   5|   1|   5|   5|   5|    5|
|      11|   4|   1|   5|   1|   5|   3|   1|   5|   5|    5|
|      12|   5|   1|   5|   1|   4|   1|   5|   5|   5|    5|
|      1

'Total count= 1015341'

In [5]:
df_ext.write.mode("overwrite").format("jdbc") \
        .option("user","root") \
        .option("password", "1664") \
        .option("url","jdbc:mysql://localhost:3306/DB") \
        .option("dbtable","Staging_ext") \
        .save()


df_extread=spark.read.format("jdbc")\
        .options(url ="jdbc:mysql://localhost:3306/DB",dbtable="Staging_ext",user="root",password="1664")\
            .load()

df_extread.show()

23/08/15 00:00:26 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+----+----+----+----+----+----+----+----+----+-----+
|UniqueID|EXT1|EXT2|EXT3|EXT4|EXT5|EXT6|EXT7|EXT8|EXT9|EXT10|
+--------+----+----+----+----+----+----+----+----+----+-----+
|       0|   4|   1|   5|   2|   5|   1|   5|   2|   4|    1|
|       1|   3|   5|   3|   4|   3|   3|   2|   5|   1|    5|
|       2|   2|   3|   4|   4|   3|   2|   1|   3|   2|    5|
|       3|   2|   2|   2|   3|   4|   2|   2|   4|   1|    4|
|       4|   3|   3|   3|   3|   5|   3|   3|   5|   3|    4|
|       5|   3|   3|   4|   2|   4|   2|   2|   3|   3|    4|
|       6|   4|   3|   4|   3|   3|   3|   5|   3|   4|    3|
|       7|   3|   1|   5|   2|   5|   2|   5|   2|   3|    2|
|       8|   2|   2|   3|   3|   4|   2|   2|   2|   4|    4|
|       9|   1|   5|   3|   5|   2|   3|   2|   4|   5|    4|
|      10|   3|   3|   2|   3|   3|   2|   4|   3|   3|    5|
|      11|   3|   1|   5|   3|   5|   1|   5|   5|   5|    3|
|      12|   4|   1|   5|   4|   5|   1|   4|   1|   5|    2|
|      1

In [6]:
df_csn.write.mode("overwrite").format("jdbc") \
        .option("user","root") \
        .option("password", "1664") \
        .option("url","jdbc:mysql://localhost:3306/DB") \
        .option("dbtable","Staging_csn") \
        .save()


df_csnread=spark.read.format("jdbc")\
        .options(url ="jdbc:mysql://localhost:3306/DB",dbtable="Staging_csn",user="root",password="1664")\
            .load()

df_csnread.show()

+--------+----+----+----+----+----+----+----+----+----+-----+
|UniqueID|CSN1|CSN2|CSN3|CSN4|CSN5|CSN6|CSN7|CSN8|CSN9|CSN10|
+--------+----+----+----+----+----+----+----+----+----+-----+
|       0|   3|   4|   3|   2|   2|   4|   4|   2|   4|    4|
|       1|   3|   2|   5|   3|   3|   1|   3|   3|   5|    3|
|       2|   4|   2|   2|   2|   3|   3|   4|   2|   4|    2|
|       3|   2|   4|   4|   4|   1|   2|   2|   3|   1|    4|
|       4|   5|   1|   5|   1|   3|   1|   5|   1|   5|    5|
|       5|   3|   2|   4|   1|   3|   2|   4|   3|   4|    3|
|       6|   3|   2|   4|   2|   1|   4|   4|   2|   2|    5|
|       7|   5|   1|   5|   1|   4|   2|   3|   2|   5|    5|
|       8|   3|   3|   4|   0|   3|   4|   1|   2|   1|    4|
|       9|   2|   5|   1|   5|   1|   4|   3|   4|   2|    2|
|      10|   1|   5|   5|   5|   1|   5|   3|   1|   1|    5|
|      11|   5|   5|   5|   3|   5|   5|   5|   3|   3|    5|
|      12|   2|   4|   5|   4|   2|   3|   1|   4|   1|    2|
|      1

In [7]:
df_arg.write.mode("overwrite").format("jdbc") \
        .option("user","root") \
        .option("password", "1664") \
        .option("url","jdbc:mysql://localhost:3306/DB") \
        .option("dbtable","Staging_arg") \
        .save()


df_argread=spark.read.format("jdbc")\
        .options(url ="jdbc:mysql://localhost:3306/DB",dbtable="Staging_arg",user="root",password="1664")\
            .load()

df_argread.show()

+--------+----+----+----+----+----+----+----+----+----+-----+
|UniqueID|AGR1|AGR2|AGR3|AGR4|AGR5|AGR6|AGR7|AGR8|AGR9|AGR10|
+--------+----+----+----+----+----+----+----+----+----+-----+
|       0|   2|   5|   2|   4|   2|   3|   2|   4|   3|    4|
|       1|   1|   4|   1|   5|   1|   5|   3|   4|   5|    3|
|       2|   1|   4|   1|   4|   2|   4|   1|   4|   4|    3|
|       3|   2|   4|   3|   4|   2|   4|   2|   4|   3|    4|
|       4|   1|   5|   1|   5|   1|   3|   1|   5|   5|    3|
|       5|   2|   3|   1|   4|   2|   3|   2|   3|   4|    4|
|       6|   1|   2|   1|   5|   3|   5|   3|   4|   4|    5|
|       7|   2|   5|   2|   4|   3|   2|   2|   4|   4|    4|
|       8|   1|   5|   1|   5|   1|   3|   1|   4|   4|    4|
|       9|   5|   3|   5|   1|   5|   3|   4|   2|   3|    2|
|      10|   2|   5|   3|   4|   2|   3|   1|   3|   4|    2|
|      11|   2|   5|   5|   3|   3|   5|   3|   5|   5|    5|
|      12|   1|   5|   2|   5|   1|   4|   1|   5|   5|    5|
|      1

In [8]:
df_opn.write.mode("overwrite").format("jdbc") \
        .option("user","root") \
        .option("password", "1664") \
        .option("url","jdbc:mysql://localhost:3306/DB") \
        .option("dbtable","Staging_opn") \
        .save()


df_opnread=spark.read.format("jdbc")\
        .options(url ="jdbc:mysql://localhost:3306/DB",dbtable="Staging_opn",user="root",password="1664")\
            .load()

df_opnread.show()

+--------+----+----+----+----+----+----+----+----+----+-----+
|UniqueID|OPN1|OPN2|OPN3|OPN4|OPN5|OPN6|OPN7|OPN8|OPN9|OPN10|
+--------+----+----+----+----+----+----+----+----+----+-----+
|       0|   5|   1|   4|   1|   4|   1|   5|   3|   4|    5|
|       1|   1|   2|   4|   2|   3|   1|   4|   2|   5|    3|
|       2|   5|   1|   2|   1|   4|   2|   5|   3|   4|    4|
|       3|   4|   2|   5|   2|   3|   1|   4|   4|   3|    3|
|       4|   5|   1|   5|   1|   5|   1|   5|   3|   5|    5|
|       5|   5|   1|   5|   1|   3|   1|   5|   4|   5|    2|
|       6|   5|   2|   4|   3|   4|   1|   5|   5|   4|    4|
|       7|   4|   1|   3|   1|   5|   1|   4|   3|   4|    5|
|       8|   4|   1|   5|   1|   4|   1|   5|   2|   5|    5|
|       9|   3|   1|   3|   1|   3|   3|   4|   3|   3|    3|
|      10|   5|   1|   5|   1|   5|   1|   5|   5|   5|    5|
|      11|   4|   1|   5|   1|   5|   3|   1|   5|   5|    5|
|      12|   5|   1|   5|   1|   4|   1|   5|   5|   5|    5|
|      1